<a href="https://colab.research.google.com/github/bok-h22/TIL-202303/blob/master/%EC%B5%9C%EC%A2%85_%EB%94%A5%EB%9F%AC%EB%8B%9D_%EB%AA%A8%EB%8D%B8%EB%A7%81_%EC%A7%80%EC%9C%A4__%ED%9D%AC%EC%A4%80_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 기본 세팅

In [ ]:
!pip install tokenizers
!pip install sentencepiece
!pip install focal_loss
!pip install scikit-learn --upgrade -user
!pip install keras --upgrade -user
!pip install tensorflow --upgrade -user
!pip install pickle --upgrade -user

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.5 MB/s eta 0:00:00

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook

import tensorflow as tf
import sentencepiece as spm

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# 데이터 불러오기

1. 기업 리뷰 데이터

In [ ]:
# input/txt
txt_raw = pd.read_csv('/content/drive/MyDrive/한스펠 돌린 최종본.csv', encoding = 'utf-8')

txt_raw= txt_raw.drop(txt_raw[txt_raw['cleand_up'].isna() == True].index)
txt_raw= txt_raw.drop(txt_raw[txt_raw['cleand_down'].isna() == True].index)
#data['label'] = data['label'].astype('str')

txt_raw = txt_raw[['corp', 'stock_code', 'year', 'rank', 'label', 'cleand_up', 'cleand_down']]

2. 재무데이터 (Model A)

In [ ]:
# input/finacial data
finalcial_raw = pd.read_csv('/content/drive/MyDrive/final_a_df.csv', encoding = 'cp949')

finalcial_raw = finalcial_raw[['corp', 'stock_code', 'year', 'rank',
       'ebitda_margin', 'ebitda_to_interest_expense', 'debt_ratio',
       'dependence_on_net_borrowings', 'net_borrowings_to_ebitda', 'revenue',
       'cogs', 'selling_general_administrative_expenses', 'ebit',
       'ebit_margin', 'ebitda_to_sales_revenue', 'total_assets',
       'return_on_assets', 'ebitda', 'financial_expenses', 'corporate_tax',
       'operating_cash_flow', 'free_cash_flow', 'total_liabilities',
       'total_equity', 'total_borrowings', 'net_borrowings',
       'borrowing_dependency', 'total_borrowings_to_ebitda',
       'debt_to_net_income_ratio', 'total_assets_leverage',
       'current_liabilities', 'working_capital', 'current_liabilities_ratio',
       'quick_assets', 'quick_ratio', 'cash_and_cash_equivalents',
       'short_term_borrowings', 'days_sales_outstanding',
       'market_capitalization', 'minimum_wage', 'us_kor_exchange_avg',
       'ppi_year', 'kor_usa_ir_diff', 'kr_standard_yield', 'count', 'rating',
       'paywellfare', 'worklifebal', 'culture', 'opportunity', 'manager',
       'recommend', 'ceo', 'potential', 'crb_index_avg']]

finalcial_raw = finalcial_raw[finalcial_raw['rank'].isna() == False]

In [ ]:
# merge
total_raw = txt_raw.merge(finalcial_raw, on = ['corp', 'stock_code', 'year', 'rank'], how = 'inner')
len(total_raw)

30463

# 데이터 분할

In [ ]:
# train/test split
from sklearn.model_selection import train_test_split

data_train, data_test = train_test_split(total_raw, test_size=0.25, stratify=total_raw['label'].to_list())
data_train, data_valid = train_test_split(data_train, test_size=0.2, stratify=data_train['label'].to_list())

# 데이터 전처리

1. 텍스트 데이터

In [ ]:
# 사전 작성
with open('test_review.txt', 'a', encoding='utf8') as f:
    f.write('\n'.join(data_train['cleand_up']))
    f.write('\n'.join(data_train['cleand_down']))
    f.write('\n'.join(data_test['cleand_up']))
    f.write('\n'.join(data_test['cleand_down']))

In [ ]:
# 토크나이즈
from tokenizers import SentencePieceBPETokenizer

tokenizer = SentencePieceBPETokenizer()
tokenizer.train('test_review.txt', vocab_size=5000, min_frequency=3)

# train
data_train['sp_up_train'] = data_train['cleand_up'].apply(lambda x: tokenizer.encode(x).ids)
data_train['sp_down_train'] = data_train['cleand_down'].apply(lambda x: tokenizer.encode(x).ids)

#valid
data_valid['sp_up_valid'] = data_valid['cleand_up'].apply(lambda x: tokenizer.encode(x).ids)
data_valid['sp_down_valid'] = data_valid['cleand_down'].apply(lambda x: tokenizer.encode(x).ids)

# test
data_test['sp_up_test'] = data_test['cleand_up'].apply(lambda x: tokenizer.encode(x).ids)
data_test['sp_down_test'] = data_test['cleand_down'].apply(lambda x: tokenizer.encode(x).ids)

# output
tokenized_up_train = data_train['sp_up_train']
tokenized_up_valid = data_valid['sp_up_valid']
tokenized_up_test = data_test['sp_up_test']

tokenized_down_train = data_train['sp_down_train']
tokenized_down_valid = data_valid['sp_down_valid']
tokenized_down_test = data_test['sp_down_test']

In [ ]:
# 패딩
max_length = 50

# train
sequences_up_train = pad_sequences(tokenized_up_train, maxlen=max_length, padding='post', truncating='post')
sequences_down_train = pad_sequences(tokenized_up_train, maxlen=max_length, padding='post', truncating='post')
labels_train = np.array(data_train['label'])

# valid
sequences_up_valid = pad_sequences(tokenized_up_valid, maxlen=max_length, padding='post', truncating='post')
sequences_down_valid = pad_sequences(tokenized_up_valid, maxlen=max_length, padding='post', truncating='post')
labels_valid = np.array(data_valid['label'])

# test
sequences_up_test = pad_sequences(tokenized_up_test, maxlen=max_length, padding='post', truncating='post')
sequences_down_test = pad_sequences(tokenized_up_test, maxlen=max_length, padding='post', truncating='post')
labels_test = np.array(data_test['label'])

In [ ]:
set(labels_valid)

{0, 1, 2, 3, 4, 5, 6, 7, 8}

2. 재무데이터

In [ ]:
# 표준화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

columns = ['ebitda_margin', 'ebitda_to_interest_expense',
       'debt_ratio', 'dependence_on_net_borrowings',
       'net_borrowings_to_ebitda', 'revenue', 'cogs',
       'selling_general_administrative_expenses', 'ebit', 'ebit_margin',
       'ebitda_to_sales_revenue', 'total_assets', 'return_on_assets', 'ebitda',
       'financial_expenses', 'corporate_tax', 'operating_cash_flow',
       'free_cash_flow', 'total_liabilities', 'total_equity',
       'total_borrowings', 'net_borrowings', 'borrowing_dependency',
       'total_borrowings_to_ebitda', 'debt_to_net_income_ratio',
       'total_assets_leverage', 'current_liabilities', 'working_capital',
       'current_liabilities_ratio', 'quick_assets', 'quick_ratio',
       'cash_and_cash_equivalents', 'short_term_borrowings',
       'days_sales_outstanding',
       'market_capitalization', 'minimum_wage', 'us_kor_exchange_avg',
       'ppi_year', 'kor_usa_ir_diff', 'kr_standard_yield', 'count', 'rating',
       'paywellfare', 'worklifebal', 'culture', 'opportunity', 'manager',
       'recommend', 'ceo', 'potential', 'crb_index_avg']

print(len(columns))

# train
scaler.fit(data_train[columns])
scaled_data_train = scaler.transform(data_train[columns])

# valid
scaled_data_valid = scaler.transform(data_valid[columns])

# test
scaled_data_test = scaler.transform(data_test[columns])

# export scaler
import pickle

with open("scaler.pkl", "wb") as file:
    pickle.dump(scaler, file)

51


# 딥러닝

1. 모델 구축

In [ ]:
import tensorflow as tf

print(tf.__version__)

import keras

print(keras.__version__)

2.12.0
2.12.0


In [ ]:
# 패키지
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, GRU, Dense, Dropout, BatchNormalization, concatenate
from focal_loss import SparseCategoricalFocalLoss
from tensorflow.keras.models import Model

# GRU(Gated Recurrent Unit)
def review_GRU():
  input_layer = Input(shape=(max_length,))
  embedding_layer = Embedding(input_dim=len(tokenizer.get_vocab()), output_dim=512, input_length=max_length)(input_layer)
  gru_1 = GRU(256, activation='tanh', return_sequences=True)(embedding_layer)
  gru_2 = GRU(128, activation='tanh')(gru_1)

  return input_layer, gru_2

# MLP(Multi-Layer Perceptron)
def fs_MLP():
  input_layer = Input(shape=[51,])
  mlp_1 = Dense(32, activation='relu')(input_layer)
  return input_layer, mlp_1

# 모델 정의
input_layer_up, output_up = review_GRU()
input_layer_down, output_down = review_GRU()
input_layer_fs, output_fs = fs_MLP()

# 위, 아래 방향 리뷰 결과와 벡터 결과를 concatenate하여 하나의 벡터로 만듭니다.
concat_input = concatenate([output_up, output_down, output_fs], axis=-1)

# Dropout과 BatchNormalization을 거쳐 은닉층 구성
dropout_1 = Dropout(0.2)(concat_input)
batchnorm_1 = BatchNormalization()(dropout_1)
dense_1 = Dense(64, activation='relu')(batchnorm_1)

dropout_2 = Dropout(0.2)(dense_1)
batchnorm_2 = BatchNormalization()(dropout_2)
dense_2 = Dense(32, activation='relu')(batchnorm_2)

# Output layer: 최종 출력을 위해 Softmax 활성화 함수가 적용된 Dense 레이어를 사용
output_layer = Dense(9, activation='softmax')(dense_2)

# Create the functional model: 위에서 정의한 입력 레이어와 출력 레이어로 모델 생성
model = Model(inputs=[input_layer_up, input_layer_down, input_layer_fs], outputs=output_layer)

# Create an optimizer with a custom learning rate
learning_rate = 0.0001
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Focal Loss: SparseCategoricalFocalLoss(gamma=2)를 사용하여 커스텀 Loss 함수 생성
loss_func = SparseCategoricalFocalLoss(gamma=2)

# Early Stopping 콜백 정의
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True
)

# Compile the model: 옵티마이저, Loss 함수, 평가 메트릭 설정
model.compile(optimizer=optimizer, loss=loss_func, metrics=['accuracy'])

# 모델 아키텍처 출력
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 50, 512)      2560000     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 50, 512)      2560000     ['input_2[0][0]']                
                                                                                              

2. 학습

In [ ]:
history = model.fit([sequences_up_train, sequences_down_train, scaled_data_train],
                    labels_train,
                    validation_data=([sequences_up_valid, sequences_down_valid, scaled_data_valid], labels_valid),
                    verbose= 1,
                    epochs=100,
                    batch_size=64,
                    callbacks=[early_stopping_callback])

Epoch 1/100
286/286 [==============================] - 52s 133ms/step - loss: 1.8877 - accuracy: 0.1757 - val_loss: 1.5870 - val_accuracy: 0.1724
Epoch 2/100
286/286 [==============================] - 14s 48ms/step - loss: 1.6069 - accuracy: 0.2197 - val_loss: 1.4060 - val_accuracy: 0.2963
Epoch 3/100
286/286 [==============================] - 9s 31ms/step - loss: 1.3453 - accuracy: 0.2981 - val_loss: 1.2412 - val_accuracy: 0.3206
Epoch 4/100
286/286 [==============================] - 8s 26ms/step - loss: 1.1188 - accuracy: 0.3904 - val_loss: 1.0464 - val_accuracy: 0.5118
Epoch 5/100
286/286 [==============================] - 9s 33ms/step - loss: 0.9641 - accuracy: 0.4557 - val_loss: 0.8418 - val_accuracy: 0.4919
Epoch 6/100
286/286 [==============================] - 8s 28ms/step - loss: 0.8523 - accuracy: 0.5029 - val_loss: 0.6616 - val_accuracy: 0.6002
Epoch 7/100
286/286 [==============================] - 8s 29ms/step - loss: 0.7577 - accuracy: 0.5523 - val_loss: 0.6737 - val_accura

3. 검증

In [ ]:
predict_test = model.predict([sequences_up_test, sequences_down_test, scaled_data_test])

predicted_classes = np.argmax(predict_test, axis=1)

238/238 [==============================] - 2s 5ms/step


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

# 예측 결과(predicted_labels)와 실제 레이블(true_labels) 비교
accuracy = accuracy_score(labels_test, predicted_classes)
f1_score = f1_score(labels_test, predicted_classes, average = 'weighted')

print(f'accuracy : {accuracy:.3f}\nF1-score : {f1_score:.3f}')

accuracy : 0.908
F1-score : 0.908


In [ ]:
from keras.models import load_model
model.save('20230727_final_model.h5')

In [ ]:
!pip install keras.utils

  Preparing metadata (setup.py) ... done
  Created wheel for keras.utils: filename=keras_utils-1.0.13-py3-none-any.whl size=2634 sha256=a30de77a66253546f9379ba310dd4d31c8f013d14a17645bf0151be1da4d1b60
  Stored in directory: /root/.cache/pip/wheels/5c/c0/b3/0c332de4fd71f3733ea6d61697464b7ae4b2b5ff0300e6ca7a
Successfully built keras.utils
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3099, in _parsed_pkg_info
    return self._pkg_info
  File "/usr

In [ ]:
import keras
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

%matplotlib inline

# 시각화할 모델의 구조를 가져옵니다.
model_structure = model_to_dot(model, show_shapes=True)

# SVG 형식으로 이미지를 생성합니다.
svg_image = SVG(model_structure.create(prog='dot', format='svg'))

display(svg_image)

# 모델 구조를 이미지 파일로 저장합니다.
plot_model(model, show_shapes=True, to_file='model_structure.png')


NameError: ignored